来源：https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f

## Sequential Model-Based Optimization（SMBO）

SMBO是一个Bayesian optimization算法。‘Sequential’意味着一轮一轮地进行尝试，每轮利用当前模型给出的最优超参数，然后利用这个超参数带入函数后验证的结果来更新模型。

这个算法包含5个组成成分：  
> 1. 超参数的定义域  
> 2. 目标函数，可以接受超参数输入，返回得分  
> 3. 目标函数对应的模型（surrogate model）  
> 4. 选择函数，用于从模型中选择最优超参数  
> 5. 历史，（超参数，得分）对，用于更新surrogate model 

hyperopt库实现了TPE算法，一个简单的例子如下：

In [2]:
from hyperopt import fmin, tpe, hp
best = fmin(
    fn=lambda x: (x-1)**2,              # 要最小化的函数，可以是任何返回实值的函数
    space=hp.uniform('x', -2, 2),       # 超参数搜索域
    algo=tpe.suggest,                   # 使用TPE算法
    max_evals=100)                      # 最大evaluation次数
print(best)

{'x': 1.0066432353098484}


hyperopt提供了一些参数的先验分布，如choice，normal，uniform，lognormal等：

In [4]:
import hyperopt.pyll.stochastic

space = {
    'x': hp.uniform('x', 0, 1),
    'y': hp.normal('y', 0, 1),
    'name': hp.choice('name', ['alice', 'bob']),
}

print(hyperopt.pyll.stochastic.sample(space))

{'name': 'bob', 'x': 0.9821704856681024, 'y': -0.43218336300612803}


hyperopt的Trails对象记录了每次迭代的细节：

In [6]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

fspace = {
    'x': hp.uniform('x', -5, 5)
}

def f(params):
    x = params['x']
    val = x**2
    return {'loss': val, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=f, space=fspace, algo=tpe.suggest, max_evals=50, trials=trials)

print('best:', best)

print('trials:')
for trial in trials.trials[:2]:
    print(trial)

best: {'x': -0.015546488490608468}
trials:
{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': 0.9492262172484285, 'status': 'ok'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'x': [0]}, 'vals': {'x': [-0.974282411443637]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2018, 11, 21, 9, 59, 50, 212000), 'refresh_time': datetime.datetime(2018, 11, 21, 9, 59, 50, 212000)}
{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': 0.7038667988805063, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'x': [1]}, 'vals': {'x': [-0.8389676983534624]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2018, 11, 21, 9, 59, 50, 213000), 'refresh_time': datetime.datetime(2018, 11, 21, 9, 59, 50, 213000)}


上面的‘tid’指time id，即time step。‘vals’中的‘x’就是本轮迭代的参数值，‘result’中的‘loss’就是本轮的目标函数值。

可以利用这些信息画出x和loss随轮数的变化图。

### 一个例子

下面的例子使用random forest来对iris数据集进行分类，超参数的选择利用hyperopt。

In [12]:
# 载入数据集

from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

In [24]:
# random forest的超参数空间

space4rf = {
    'max_depth': hp.choice('max_depth', range(1,20)),
    'max_features': hp.choice('max_features', range(1,5)),
    'n_estimators': hp.choice('n_estimators', range(1,20)),
    'criterion': hp.choice('criterion', ["gini", "entropy"]),
    'scale': hp.choice('scale', [0, 1]),
    'normalize': hp.choice('normalize', [0, 1])
}

In [34]:
# 目标函数，输入超参数返回得分

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize,scale    

def hyperopt_train_test(params):
    X_ = X[:]
    if 'normalize' in params:
        if params['normalize'] == 1:
            X_ = normalize(X_)        # 将输入向量调整到相同的（l2）范数
        del params['normalize']

    if 'scale' in params:
        if params['scale'] == 1:
            X_ = scale(X_)            # 将输入向量调整到均值为0，方差为1
        del params['scale']
            
    clf = RandomForestClassifier(**params)
    
    return cross_val_score(clf, X, y, cv=5).mean()

In [35]:
best = 0

def f(params):
    global best
    acc = hyperopt_train_test(params)
    if acc > best:
        best = acc
    print('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

In [36]:
trials = Trials()
best = fmin(f, space4rf, algo=tpe.suggest, max_evals=300, trials=trials)
print('best parameters:')
print(best)

new best: 0.9533333333333334 {'criterion': 'entropy', 'max_depth': 5, 'max_features': 2, 'n_estimators': 17}
new best: 0.9533333333333334 {'criterion': 'gini', 'max_depth': 6, 'max_features': 1, 'n_estimators': 11}
new best: 0.9533333333333334 {'criterion': 'gini', 'max_depth': 1, 'max_features': 4, 'n_estimators': 6}
new best: 0.9533333333333334 {'criterion': 'entropy', 'max_depth': 11, 'max_features': 1, 'n_estimators': 6}
new best: 0.9666666666666668 {'criterion': 'gini', 'max_depth': 4, 'max_features': 3, 'n_estimators': 13}
new best: 0.9666666666666668 {'criterion': 'entropy', 'max_depth': 5, 'max_features': 1, 'n_estimators': 12}
new best: 0.9666666666666668 {'criterion': 'gini', 'max_depth': 16, 'max_features': 4, 'n_estimators': 3}
new best: 0.9666666666666668 {'criterion': 'entropy', 'max_depth': 17, 'max_features': 4, 'n_estimators': 15}
new best: 0.9666666666666668 {'criterion': 'gini', 'max_depth': 14, 'max_features': 1, 'n_estimators': 19}
new best: 0.9666666666666668 {'cr

new best: 0.9666666666666668 {'criterion': 'entropy', 'max_depth': 4, 'max_features': 3, 'n_estimators': 13}
new best: 0.9666666666666668 {'criterion': 'gini', 'max_depth': 4, 'max_features': 2, 'n_estimators': 3}
new best: 0.9666666666666668 {'criterion': 'entropy', 'max_depth': 14, 'max_features': 3, 'n_estimators': 12}
new best: 0.9666666666666668 {'criterion': 'entropy', 'max_depth': 15, 'max_features': 2, 'n_estimators': 17}
new best: 0.9666666666666668 {'criterion': 'gini', 'max_depth': 17, 'max_features': 2, 'n_estimators': 17}
new best: 0.9666666666666668 {'criterion': 'gini', 'max_depth': 19, 'max_features': 4, 'n_estimators': 14}
new best: 0.9666666666666668 {'criterion': 'entropy', 'max_depth': 19, 'max_features': 4, 'n_estimators': 18}
new best: 0.9666666666666668 {'criterion': 'entropy', 'max_depth': 8, 'max_features': 2, 'n_estimators': 17}
new best: 0.9666666666666668 {'criterion': 'entropy', 'max_depth': 10, 'max_features': 2, 'n_estimators': 19}
new best: 0.96666666666

new best: 0.9733333333333334 {'criterion': 'entropy', 'max_depth': 10, 'max_features': 3, 'n_estimators': 6}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 12, 'max_features': 4, 'n_estimators': 12}
new best: 0.9733333333333334 {'criterion': 'entropy', 'max_depth': 2, 'max_features': 2, 'n_estimators': 19}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 12, 'max_features': 4, 'n_estimators': 15}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 9, 'max_features': 2, 'n_estimators': 19}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 14, 'max_features': 3, 'n_estimators': 7}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 14, 'max_features': 4, 'n_estimators': 12}
new best: 0.9733333333333334 {'criterion': 'entropy', 'max_depth': 15, 'max_features': 1, 'n_estimators': 18}
new best: 0.9733333333333334 {'criterion': 'entropy', 'max_depth': 15, 'max_features': 4, 'n_estimators': 17}
new best: 0.9733333333333334 

new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 15, 'max_features': 4, 'n_estimators': 11}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 15, 'max_features': 2, 'n_estimators': 11}
new best: 0.9733333333333334 {'criterion': 'entropy', 'max_depth': 19, 'max_features': 4, 'n_estimators': 2}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 10, 'max_features': 4, 'n_estimators': 19}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 11, 'max_features': 4, 'n_estimators': 14}
new best: 0.9733333333333334 {'criterion': 'entropy', 'max_depth': 6, 'max_features': 4, 'n_estimators': 5}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 6, 'max_features': 3, 'n_estimators': 5}
new best: 0.9733333333333334 {'criterion': 'entropy', 'max_depth': 13, 'max_features': 3, 'n_estimators': 7}
new best: 0.9733333333333334 {'criterion': 'gini', 'max_depth': 1, 'max_features': 1, 'n_estimators': 16}
new best: 0.9733333333333334 {'crit